# Mass reconstruction posibilities

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from tqdm import tqdm
import swordfish as sf
from WIMpy import DMUtils as DMU
from scipy.interpolate import interp1d
from scipy.integrate import cumtrapz
from scipy.integrate import cumtrapz
from paleo.paleopy import *

rc('text', usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['cmr']})
rc('font',**{'family':'serif','serif':['cmr']})
%matplotlib inline

#### These are the bins we are considering - Change here

In [7]:
x_bins = np.linspace(1,1000,67)
x_width = np.diff(x_bins)
x_c = x_bins[:-1] + x_width/2
E_list = np.logspace(-1, 3, 500) # keV

#### The region I will consider is between the 95% CL and three orders of magnitude above that. This roughly corresponds to the current sensitivity of Xenon detectors.